# Setup
1. install python (3.8) and git
2. install virtualenv e.g. with `<path to python> -m to pip install virtualenv`
3. create a virtual environment that uses python 3.8 using virtualenv e.g. with `<path to python> -m virtualenv "<path to virtual environment>"`
4. activate the virtual environment e.g. on windows with `.\<pathto env>\Scripts\activate`
5. install vscode and python extension
6. git clone brainlit (https://github.com/neurodata/brainlit) and neuroglancer (https://github.com/google/neuroglancer)
7. install brainlit from source e.g. `pip install -e .` in brainlit folder 
8. Place `test-czi.zip` file in `brainlit/experiments/sriramg/data/` and unzip it

In [ ]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr, write_trace_layer
import zarr
from cloudvolume import CloudVolume
import json
from pathlib import Path

# Convert czi to zarr

In [ ]:
project_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi"  # "E:\\Projects\\KolodkinLab\\Sriram\\brainlit-tracing\\brainlit\\experiments\\sriram\\data\\" "C:\\Users\\Sriram Sudarsanam\\Desktop\\NeuroglancerTrial\\"
project_path = Path(project_path)

czi_path = project_path / "test.czi"  # path to czi image

In [ ]:
zarr_paths = czi_to_zarr(
    czi_path=czi_path, out_dir=project_path, fg_channel=1, parallel=1
)

# Convert zarr to ome-zarr and add trace layer

In [ ]:
ome_path = project_path / "fg_ome.zarr"  # path of ome zarr to be made
resolution = [510, 510, 3000]  # xyz resolution in nm

In [ ]:
zarr_to_omezarr(zarr_path=zarr_paths[0], out_path=ome_path, res=resolution)
write_trace_layer(parent_dir=project_path, res=resolution)

# View ome-zarr on neuroglancer

1. Use https://github.com/google/neuroglancer/blob/master/cors_webserver.py to serve the data e.g. `python cors_webserver.py -d "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data" -p 9010`
2. Launch neuroglancer app e.g. https://neuroglancer-demo.appspot.com
3. Add layer to neuroglancer app e.g. zarr://http://127.0.0.1:9010/sriram-adipo-brain1-im3/fg_ome.zarr

# Ready to trace
Use `trace.py` to trace neurons

# Optional: Activate ViterBrain

You will need to download ilastik: https://www.ilastik.org/download.html

Update `ilastik_program_path` below, e.g. for Mac/Linux use the path to run_ilastik.sh, for Windows, use the path to ilastik.exe

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation

In [ ]:
ilastik_program_path = "/Applications/ilastik-1.4.0b21-OSX.app/Contents/ilastik-release/run_ilastik.sh"  # path to ilastik executable e.g. for windows something like "\Program Files\ilastik-1.3.2\ilastik.exe"


image_path = project_path / "fg_ome.zarr" / "0"
z_im = zarr.open_array(image_path)
ilastik_project_path = project_path / "ex_fg.ilp"
chunk_size = [c * 2 for c in z_im.chunks]
data_bin = project_path / "data_bin/"

prob_path = project_path / "probs.zarr"
fragment_path = project_path / "labels.zarr"
ome_path_lbl = project_path / "labels_ome.zarr"
tiered_path = project_path / "tiered.zarr"
states_path = project_path / "nx.pickle"

sg = state_generation(
    image_path=image_path,
    new_layers_dir=project_path,
    ilastik_program_path=ilastik_program_path,
    ilastik_project_path=ilastik_project_path,
    chunk_size=chunk_size,
    parallel=6,
    resolution=[
        resolution[2] / 1000,
        resolution[0] / 1000,
        resolution[1] / 1000,
    ],  # ome zarr is zxy, in microns
    # prob_path=prob_path,
    # fragment_path=fragment_path,
    # tiered_path=tiered_path,
    # states_path=states_path,
)

In [ ]:
sg.predict(data_bin=data_bin)
sg.compute_frags()

zarr_to_omezarr(zarr_path=fragment_path, out_path=ome_path_lbl, res=resolution)

In [ ]:
sg.compute_soma_lbls()
sg.compute_image_tiered()
sg.compute_states()
sg.compute_edge_weights(str(ome_path_lbl / "0"))

# Other

In [ ]:
from skimage import io
import numpy as np


image_path = project_path / "fg_ome.zarr" / "0"
z_im = zarr.open(image_path)
subvol = z_im[:40, :40, :40]
io.imsave(project_path / "train.tif", subvol)

In [ ]:
import pickle
from brainlit.algorithms.connect_fragments.viterbrain import explain_viterbrain

with open(
    "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/test-czi/viterbrain.pickle",
    "rb",
) as handle:
    vb = pickle.load(handle)

In [ ]:
fragments = zarr.open(vb.fragment_path, mode="r")

In [ ]:
explain_viterbrain(vb, [558, 549, 4], [480, 517, 2], [])

vs fragments [1176, 1173, 1172]

# Create Viterbrain object

In [ ]:
from brainlit.algorithms.generate_fragments.state_generation import state_generation
import numpy as np
import os

In [ ]:
im0 = zarr.open(f"{out_path}/0/")
probs_ra = np.zeros(im0.shape)
probs_ra[np.array(im0) > 20] = 0.95


prob_path = f"{project_path}probs.zarr"
probs = zarr.open(prob_path, mode="w", shape=im0.shape)
probs[:,:,:] = probs_ra

In [ ]:
sg = state_generation(
    image_path=f"{out_path}/0/",
    new_layers_dir=project_path,
    ilastik_program_path="",
    ilastik_project_path="",
    fg_channel=0,
    chunk_size=[300, 300, 160],
    resolution=[0.5, 0.5, 3],
    parallel=8,
    prob_path = prob_path,
)


sg.compute_frags()
sg.compute_image_tiered()
sg.compute_soma_lbls()
sg.compute_states()
sg.compute_edge_weights()

In [ ]:
labels_path = f"{project_path}labels.zarr"
labels_ome_path = f"{project_path}labels_ome_hack.zarr"

labels = zarr.open(labels_ome_path, "w")
labels.create_dataset("0",shape = im0.shape, chunks = im0.chunks, dtype="int")
labels.attrs["multiscales"] = [
        {
            "axes": [
                {
                    "name": "x",
                    "type": "space"
                },
                {
                    "name": "y",
                    "type": "space"
                },
                {
                    "name": "z",
                    "type": "space"
                }
            ],
            "datasets": [
                {
                    "coordinateTransformations": [
                        {
                            "scale": [
                                1.0,
                                1.0,
                                1.0
                            ],
                            "type": "scale"
                        }
                    ],
                    "path": "0"
                }
            ],
            "name": "/",
            "version": "0.4"
        }
    ]

In [ ]:
allfiles = os.listdir(labels_path)
 
# iterate on all files to move them to destination folder
for f in allfiles:
    src_path = os.path.join(labels_path, f)
    dst_path = os.path.join(f"{labels_ome_path}/0/", f)
    os.rename(src_path, dst_path)

os.rmdir(labels_path)